In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [ ]:
anime_df = pd.read_csv('anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
rating_df = pd.read_csv('rating.csv')
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1.0
1,1,24,-1.0
2,1,79,-1.0
3,1,226,-1.0
4,1,241,-1.0


In [ ]:
rating_df = rating_df[rating_df.user_id <= 10000]


In [ ]:
rating_df['rating'] = rating_df['rating'].replace(-1, pd.NA)


In [ ]:
rating_df = rating_df.drop_duplicates(['user_id', 'anime_id'])


In [ ]:
user_item_matrix = rating_df.pivot(index='user_id', columns='anime_id', values='rating')

In [ ]:
user_similarity = cosine_similarity(user_item_matrix.fillna(0))


<ipython-input-9-578e5d18b01d>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_similarity = cosine_similarity(user_item_matrix.fillna(0))


In [ ]:
def user_based_recommendations(user_id, user_item_matrix, user_similarity, n=5):
    # Get the similarity scores for the target user
    user_scores = user_similarity[user_id - 1]

    # Sort users by similarity in descending order
    similar_users = sorted(enumerate(user_scores), key=lambda x: x[1], reverse=True)

    # Initialize a list to store recommended anime
    recommended_anime = []

    for user, score in similar_users[1:]:  # Exclude the target user
        # Get the anime the similar user has rated positively
        rated_anime = user_item_matrix.loc[user_item_matrix.index[user - 1]]
        positively_rated_anime = rated_anime[rated_anime >= 6].index

        # Exclude anime the target user has already rated
        target_user_rated_anime = user_item_matrix.loc[user_id].dropna().index
        recommended_anime.extend(set(positively_rated_anime) - set(target_user_rated_anime))

        # Limit the number of recommendations
        if len(recommended_anime) >= n:
            break

    return recommended_anime[:n]

In [ ]:
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)


<ipython-input-11-5098c16ba4a5>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)


In [ ]:
def item_based_recommendations(user_id, user_item_matrix, item_similarity, n=5):
    # Get the anime the user has rated positively
    positively_rated_anime = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] >= 6].index

    # Initialize a list to store recommended anime
    recommended_anime = []

    for anime_id in positively_rated_anime:
        # Get anime similar to the liked anime
        similar_anime = item_similarity[anime_id - 1]

        # Sort similar anime by similarity in descending order
        similar_anime_sorted = sorted(enumerate(similar_anime), key=lambda x: x[1], reverse=True)

        for anime, score in similar_anime_sorted[1:]:  # Exclude the liked anime
            # Exclude anime the user has already rated
            user_rated_anime = user_item_matrix.loc[user_id].dropna().index
            if anime not in user_rated_anime:
                recommended_anime.append(anime)

            # Limit the number of recommendations
            if len(recommended_anime) >= n:
                break

    return recommended_anime[:n]

In [ ]:
user_id = 9
user_recommendations = user_based_recommendations(user_id, user_item_matrix, user_similarity, n=5)
item_recommendations = item_based_recommendations(user_id, user_item_matrix, item_similarity, n=5)

In [ ]:
print(f"User-Based Recommendations for User {user_id}:")
for anime_id in user_recommendations:
    anime_name = anime_df.loc[anime_df['anime_id'] == anime_id, 'name'].values[0]
    print(f"- {anime_name}")

print(f"\nItem-Based Recommendations for User {user_id}:")
for anime_id in item_recommendations:
    anime_name = anime_df.loc[anime_df['anime_id'] == anime_id, 'name'].values[0]
    print(f"- {anime_name}")

User-Based Recommendations for User 9:
- Campione!: Matsurowanu Kamigami to Kamigoroshi no Maou
- Yamada-kun to 7-nin no Majo (TV)
- Re:Zero kara Hajimeru Isekai Seikatsu
- Haiyore! Nyaruko-san
- Ore ga Ojousama Gakkou ni &quot;Shomin Sample&quot; Toshite Gets♥Sareta Ken

Item-Based Recommendations for User 9:
- Fushigiboshi no☆Futagohime Gyu!
- Tanpen Unico: Kuroi Kumo to Shiroi Hane
- Urban Square: Kouhaku no Tsuigeki
- Chikansha Thomas
- Nogizaka Haruka no Himitsu
